In [26]:
# Importing the libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error

In [27]:
# Importing the training dataset
Dataset = pd.read_csv('dodgers_training.csv')
#Dataset

#importing the testing dataset
Test_Dataset = pd.read_csv('dodgers_testing.csv')
#Test_Dataset

#importing the predicted dataset
Pred = pd.read_csv('predict.csv')
Pred

,month_3,day_of_week_0,temp,skies_0,bobblehead_0
0,1,1,72,1,0


In [28]:
#selecting the needed columns of trainng set for the analysis
#Dependant variable selection
Dependant = Dataset.loc[:,['attend']]

#Independent variable selection
Independent = Dataset.loc[:,['month','day_of_week','temp','skies','bobblehead']]

#selecting the needed columns of test set for the analysis
#Dependant variable selection
Test_Dependant = Test_Dataset.loc[:,['attend']]

#Independent variable selection
Test_Independent = Test_Dataset.loc[:,['month','day_of_week','temp','skies','bobblehead']]
#Test_Independent

Data Preprocessing; Encoding categorical data

In [29]:
#check data type of training set and transform them to codeable data type
#print(Independent.dtypes)
Independent['month'] = Independent['month'].astype('category')
Independent['day_of_week'] = Independent['day_of_week'].astype('category')
Independent['bobblehead'] = Independent['bobblehead'].astype('category')
Independent['skies'] = Independent['skies'].astype('category')
#print(Independent.dtypes)

#check data type of test set and transform them to codeable data type
Test_Independent['month'] = Test_Independent['month'].astype('category')
Test_Independent['day_of_week'] = Test_Independent['day_of_week'].astype('category')
Test_Independent['bobblehead'] = Test_Independent['bobblehead'].astype('category')
Test_Independent['skies'] = Test_Independent['skies'].astype('category')
#print(Test_Independent.dtypes)

In [30]:
#Transforming training set needed variables to codeable data type
Independent['month'] = Independent['month'].cat.codes
Independent['day_of_week'] = Independent['day_of_week'].cat.codes
Independent['bobblehead'] = Independent['bobblehead'].cat.codes
Independent['skies'] = Independent['skies'].cat.codes

#Checking few sample of the dataset to see if it is coded as expected
Independent.head()

#Transforming test set needed variables to codeable data type
Test_Independent['month'] = Test_Independent['month'].cat.codes
Test_Independent['day_of_week'] = Test_Independent['day_of_week'].cat.codes
Test_Independent['bobblehead'] = Test_Independent['bobblehead'].cat.codes
Test_Independent['skies'] = Test_Independent['skies'].cat.codes

#Checking few sample of the dataset to see if it is coded as expected
#print(Test_Independent)

In [31]:
#leveling out the training set encoded categories to numerically equal for all possibile choices  
Independent_onehot = Independent
Independent_onehot = pd.get_dummies(Independent_onehot,
                                    columns=['month',
                                             'day_of_week',
                                             'bobblehead', 
                                             'skies'], 
                                    prefix = ['month', 
                                              'day_of_week',
                                              'bobblehead', 
                                              'skies'])
#print(Independent_onehot.head())

#leveling out the test set encoded categories to numerically equal for all possibil choices
Test_Independent_onehot = Test_Independent
Test_Independent_onehot = pd.get_dummies(Test_Independent_onehot,
                                    columns=['month',
                                             'day_of_week',
                                             'bobblehead', 
                                             'skies'], 
                                    prefix = ['month', 
                                              'day_of_week',
                                              'bobblehead', 
                                              'skies'])
#print(Independent_onehot.head())

In [33]:
#Training set and Testing set shape and dimension manipulations to make them equalin size  
Diff_shape = set(Independent_onehot) - set(Test_Independent_onehot)
Diff_shape1 = pd.DataFrame(data = np.zeros((Test_Independent_onehot.shape[0], len(Diff_shape))), columns = list(Diff_shape))
Test_Independent_onehot = Test_Independent_onehot.join(Diff_shape1)

#Training set and Predicted set shape and dimension manipulations to make them equalin size
Diff_shape_p = set(Independent_onehot) - set(Pred)
Diff_shape_p = pd.DataFrame(data = np.zeros((Pred.shape[0], len(Diff_shape_p))), columns = list(Diff_shape_p))
Pred = Pred.join(Diff_shape_p)

In [34]:
#Listing the means and standard deviations of the encoded attributes before scaling
print('The standard deviation for Training set attributes are:',Independent_onehot.std())
print('The Mean for Training set attributes are:',Independent_onehot.mean())
print('The standard deviation for Test set attributes are:',Test_Independent_onehot.std())
print('The mean for Test set attributes are:',Test_Independent_onehot.mean())

The standard deviation for Training set attributes are: temp             8.949207
month_0          0.370591
month_1          0.414039
month_2          0.287736
month_3          0.287736
month_4          0.426021
month_5          0.133631
month_6          0.400892
day_of_week_0    0.386459
day_of_week_1    0.353094
day_of_week_2    0.370591
day_of_week_3    0.400892
day_of_week_4    0.287736
day_of_week_5    0.287736
day_of_week_6    0.353094
bobblehead_0     0.287736
bobblehead_1     0.287736
skies_0          0.400892
skies_1          0.400892
dtype: float64
The Mean for Training set attributes are: temp             73.642857
month_0           0.160714
month_1           0.214286
month_2           0.089286
month_3           0.089286
month_4           0.232143
month_5           0.017857
month_6           0.196429
day_of_week_0     0.178571
day_of_week_1     0.142857
day_of_week_2     0.160714
day_of_week_3     0.196429
day_of_week_4     0.089286
day_of_week_5     0.089286
day_of_week_6  

standardization of variables

In [35]:
#training data standardization (to regress the data mean to 0 and standard deviation to 1)
standardize = preprocessing.StandardScaler().fit(Independent_onehot)
Independent_onehot = standardize.transform(Independent_onehot)

#testing data standardization (to regress the data mean to 0 and standard deviation to 1)
Test_standardize = preprocessing.StandardScaler().fit(Test_Independent_onehot)
Test_Independent_onehot = Test_standardize.transform(Test_Independent_onehot)

#training data standardization for dependent (to regress the data mean to 0 and standard deviation to 1)
#D_standardize = preprocessing.StandardScaler().fit(Dependant)
#Dependant = D_standardize.transform(Dependant)

#testing data standardization for dependent (to regress the data mean to 0 and standard deviation to 1)
#T_D_standardize = preprocessing.StandardScaler().fit(Test_Dependant)
#Test_Dependant = T_D_standardize.transform(Test_Dependant)
Dependant.std()
Dependant.mean()

#standardization for predicted dataset
Pred_standardize = preprocessing.StandardScaler().fit(Pred)
Pred = Pred_standardize.transform(Pred)
Pred

C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\CHERPENS\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\CHERPENS\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConver

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]])

Ridge Regression:

Ridge() function has an alpha parameter (λ, but with a different name!) which is use to tune model. The assignment requires us to generate 100  regularization coefficients evenly spaced between 0.1 and 1000

In [36]:
Alphas = np.linspace(0.1,1000,100)
# standardize variables so that they are on the same scale and store each alpha in a matrix alpha_coefs
ridge_regression = Ridge()
alpha_coefs = []
RMSE1 = []
for ind_alpha in Alphas:
    ridge_regression.set_params(alpha = ind_alpha)
    ridge_regression.fit(Independent_onehot, Dependant)
    alpha_coefs.append(ridge_regression.coef_)
    RMSE1.append(np.sqrt(mean_squared_error(Test_Dependant, ridge_regression.predict(Test_Independent_onehot))))
np.shape(alpha_coefs)

#print('the coefficient for individual alpha are:', alpha_coefs)
#print(RMSE1)
#print(alpha_coefs)

(100, 1, 19)

But we do this using the cross-validated ridge regression function, RidgeCV() as the assignment requires of us and by default, the function performs generalized cross-validation.

In [42]:
ridgecv_regression = RidgeCV(alphas = Alphas, cv = None, store_cv_values=True)
ridgecv_regression.fit(Independent_onehot, Dependant)
RMSE_Test = np.sqrt(mean_squared_error(Test_Dependant, ridgecv_regression.predict(Test_Independent_onehot)))
RMSE_Train = np.sqrt(mean_squared_error(Dependant, ridgecv_regression.predict(Independent_onehot)))
print(ridgecv_regression.predict(Pred))
print('Test RMSE',RMSE_Test)
print('Train RMSE',RMSE_Train)
print(ridgecv_regression.alpha_)
print(ridgecv_regression.coef_)

[[39664.80357143]]
Test RMSE 9516.130738936348
Train RMSE 5648.86421638529
50.6
[[  515.85380007  -273.56102806   457.54654532  -216.88248689
   1102.57100111  -605.71237164   -75.78927596  -186.41964828
     67.69322043  -835.67182144   490.25315842   238.14165655
   -341.53918562  1070.28517076  -617.19884398 -1145.58152754
   1145.58152754   271.19412682  -271.19412682]]


The value of alpha that results in the smallest cross-validation error is 50.6 What is the test Root Mean Square Error (RMSE) associated with this value of alpha

In [38]:
ridge_alpha_min = Ridge(alpha = ridgecv_regression.alpha_)
ridge_alpha_min.fit(Independent_onehot, Dependant)
MSE = mean_squared_error(Test_Dependant, ridge_alpha_min.predict(Test_Independent_onehot))
RMSE3 = np.sqrt(MSE)
ridge_alpha_min.coef_
RMSE3

9516.130738936348

Lasso

In [39]:
lassocv_regression = LassoCV(alphas = Alphas, cv = 56)
lassocv_regression.fit(Independent_onehot, Dependant)
RMSE_Test_l1 = np.sqrt(mean_squared_error(Test_Dependant, lassocv_regression.predict(Test_Independent_onehot)))
RMSE_Train_l1 = np.sqrt(mean_squared_error(Dependant, lassocv_regression.predict(Independent_onehot)))
print(lassocv_regression.predict(Pred))
print(lassocv_regression.coef_)
print(lassocv_regression.alpha_)
print(RMSE_Test_l1)
print(RMSE_Train_l1)

C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might 

[39664.80357143]
[ 8.22025759e+02 -0.00000000e+00  7.53558875e+02  0.00000000e+00
  1.93339957e+03 -1.24747783e+02 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -1.16694120e+03  1.74206782e+02  0.00000000e+00
 -4.80108091e+02  1.38228372e+03 -8.85403532e+02 -2.64057436e+03
  8.44530795e-13  0.00000000e+00 -0.00000000e+00]
484.9
10505.395853369966
5464.527851899147


C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\CHERPENS\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [40]:
lassocv_regression.alpha_

484.9

Training a linear model without regularization

In [41]:
LinMod = LinearRegression()
LinMod.fit(Independent_onehot, Dependant)
print(LinMod.predict(Pred))
RMSE_Test_lm = np.sqrt(mean_squared_error(Test_Dependant, LinMod.predict(Test_Independent_onehot)))
RMSE_Train_lm = np.sqrt(mean_squared_error(Dependant, LinMod.predict(Independent_onehot)))
print(RMSE_Test_lm )
print(RMSE_Train_lm)
print(LinMod.coef_)

[[39664.80357143]]
9650.899914012849
5254.5387249249425
[[ 1235.03246476  -343.8027503    601.96103584    43.1056148
   2169.80356758  -807.51726458  -700.46213412  -800.55910692
    285.11158699 -1540.71334797   758.96677388   439.79660841
   -874.86349458  2171.01352368 -1123.4801632  -1406.04880772
   1406.04880772   -15.48024918    15.48024918]]
